In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from keras import metrics




**Data converted to dataframe

In [3]:
data = pd.read_csv('./spam.csv',encoding='latin-1')

In [4]:
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":'label', "v2":'text'})
tags = data["label"]
texts = data["text"]

for k in range(len(tags)):
    if (tags[k]=='ham'):
        tags[k]='normal'
    else:
        tags[k]='spam'

h=data['label']=='spam'
print("Spam sms count" ,len(data[h]))
h=data['label']=='normal'
print("Normal sms count" ,len(data[h]))

Spam sms count 747
Normal sms count 4825


**Redundant spaces cleared**

In [5]:
print (data.head())
print('***************************************************')
print('Spam Sms Example  :  ',data.text.iloc[2])
print('***************************************************')
print('Normal Sms Example  :  ',data.text.iloc[3])

    label                                               text
0  normal  Go until jurong point, crazy.. Available only ...
1  normal                      Ok lar... Joking wif u oni...
2    spam  Free entry in 2 a wkly comp to win FA Cup fina...
3  normal  U dun say so early hor... U c already then say...
4  normal  Nah I don't think he goes to usf, he lives aro...
***************************************************
Spam Sms Example  :   Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
***************************************************
Normal Sms Example  :   U dun say so early hor... U c already then say...


In [6]:
## For enumeration up to a maximum of 1000
num_max = 1000

## Tags make 0 and 1
le = LabelEncoder()
tags = le.fit_transform(tags)

## The process of enumerating words
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)

# Number of word counts
print(tok.word_docs)

{'available': 18, 'got': 228, 'great': 108, 'in': 805, 'wat': 95, 'world': 34, 'n': 124, 'amore': 1, 'point': 13, 'e': 85, 'until': 27, 'bugis': 7, 'cine': 7, 'go': 262, 'only': 210, 'buffet': 2, 'there': 185, 'crazy': 14, 'la': 7, 'jurong': 1, 'ok': 274, 'wif': 26, 'joking': 6, 'oni': 4, 'lar': 37, 'u': 820, '2': 412, 'std': 11, 'text': 174, 'apply': 32, 'rate': 32, 'txt': 165, '21st': 3, "08452810075over18's": 2, 'entry': 21, 'final': 18, 'free': 229, 'tkts': 4, 'win': 69, '87121': 4, 'to': 1680, 'wkly': 14, 'question': 22, 'may': 48, 'receive': 34, 'cup': 8, 't': 72, 'fa': 2, "c's": 18, 'a': 1169, '2005': 3, 'comp': 12, 'early': 31, 'c': 94, 'dun': 51, 'then': 232, 'hor': 2, 'say': 88, 'so': 432, 'already': 89, 'think': 129, 'he': 147, 'here': 122, 'lives': 4, 'goes': 25, 'usf': 11, 'i': 1669, 'nah': 10, 'though': 27, "don't": 119, 'around': 57, 'for': 619, 'word': 38, 'back': 147, '50': 57, 'you': 1535, 'chgs': 1, 'darling': 5, 'fun': 30, 'å£1': 39, 'send': 194, '3': 81, 'like': 22

In [7]:
# Indexing the word
print(tok.word_index)

{'i': 1, 'to': 2, 'you': 3, 'a': 4, 'the': 5, 'u': 6, 'and': 7, 'in': 8, 'is': 9, 'me': 10, 'my': 11, 'for': 12, 'your': 13, 'it': 14, 'of': 15, 'call': 16, 'have': 17, 'on': 18, '2': 19, 'that': 20, 'now': 21, 'are': 22, 'so': 23, 'but': 24, 'not': 25, 'or': 26, 'do': 27, 'can': 28, 'at': 29, "i'm": 30, 'get': 31, 'be': 32, 'will': 33, 'if': 34, 'ur': 35, 'with': 36, 'just': 37, 'no': 38, 'we': 39, 'this': 40, 'gt': 41, '4': 42, 'lt': 43, 'up': 44, 'when': 45, 'ok': 46, 'free': 47, 'from': 48, 'how': 49, 'go': 50, 'all': 51, 'out': 52, 'what': 53, 'know': 54, 'like': 55, 'good': 56, 'then': 57, 'got': 58, 'was': 59, 'come': 60, 'its': 61, 'am': 62, 'time': 63, 'only': 64, 'day': 65, 'love': 66, 'there': 67, 'send': 68, 'he': 69, 'want': 70, 'text': 71, 'as': 72, 'txt': 73, 'one': 74, 'going': 75, 'by': 76, 'home': 77, "i'll": 78, 'need': 79, 'about': 80, 'r': 81, 'lor': 82, 'sorry': 83, 'stop': 84, 'still': 85, 'see': 86, 'back': 87, 'today': 88, 'n': 89, 'da': 90, 'our': 91, 'reply':

In [8]:
# For example, how to enumerate words
print(texts[1])
print(tok.word_index['ok'],tok.word_index['lar'],tok.word_index['joking'],tok.word_index['wif'],tok.word_index['u'],tok.word_index['oni'])

Ok lar... Joking wif u oni...
46 336 1495 470 6 1929


In [9]:
## A maximum of 100 words and sentences are maintained
max_len = 100
cnn_texts_seq = tok.texts_to_sequences(texts)
for i in range(len(cnn_texts_seq)):
    if(len(cnn_texts_seq[i])>100):
        print('Word Counts:', len(cnn_texts_seq[i]),'Indeks:',i)


Word Counts: 172 Indeks: 1084
Word Counts: 143 Indeks: 1862
Word Counts: 101 Indeks: 2157
Word Counts: 101 Indeks: 2847


In [10]:
## A maximum of 100 words and sentences are maintained
## The number of words is made from 100. Missing words are written to 0.

cnn_texts_mat = sequence.pad_sequences(cnn_texts_seq,maxlen=max_len)

## There are 30 words in the second sentence.
## All words are indexed
## The most used 1000 words are taken.
## Less used words are removed.
## If the number of words is less than 100, 0 is added. 
## If the number of words is greater than 100 is deleted
print('***************************************************')
print(texts[2])
print(cnn_texts_mat[2])
print('***************************************************')


***************************************************
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0  47 486   8  19   4 796 899   2 178 658 267  71   2
   2 337 486 554 955  73 388 179 659 389]
***************************************************


In [11]:
## Number of words 101
## The word sad has been deleted.
## There are 100 words left.
print('***************************************************')
print(texts[2157])
print('***************************************************')
print(cnn_texts_mat[2157])
print('***************************************************')

print('sad index:',tok.word_index['sad'], 'story index:',tok.word_index['story'])


***************************************************
Sad story of a Man - Last week was my b'day. My Wife did'nt wish me. My Parents forgot n so did my Kids . I went to work. Even my Colleagues did not wish. As I entered my cabin my PA said, '' Happy B'day Boss !!''. I felt special. She askd me 4 lunch. After lunch she invited me to her apartment. We went there. She said,'' do u mind if I go into the bedroom for a minute ? '' ''OK'', I sed in a sexy mood. She came out 5 minuts latr wid a cake...n My Wife, My Parents, My Kidz, My Friends n My Colleagues. All screaming.. SURPRISE !! and I was waiting on the sofa.. ... ..... ' NAKED...!
***************************************************
[615  15   4 287 173 112  59  11  11 454 256  10  11 761 397  89  23 114
  11   1 216   2 142 211  11 114  25 256  72   1 971  11  11 395 159 590
 135 590   1 851 262  95 916  10  42 285 155 285  95 917  10   2 103  39
 216  67  95 159 590  27   6 319  34   1  50 360   5  12   4 644 590   1
   8   4 482  9

In [12]:
model = Sequential()
model.add(Embedding(1000,20,input_length=max_len))
model.add(Dropout(0.2))
model.add(Conv1D(128,3,padding='valid',activation='relu',strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['acc'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 20)           20000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 20)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 98, 128)           7808      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                4128      
__________

In [13]:
history=model.fit(cnn_texts_mat,tags,batch_size=32,epochs=10,verbose=1,validation_split=0.2)

Train on 4457 samples, validate on 1115 samples
Epoch 1/10
4457/4457 [==============================] - 5s - loss: 0.3379 - acc: 0.8782 - val_loss: 0.1672 - val_acc: 0.9561
Epoch 2/10
4457/4457 [==============================] - 5s - loss: 0.0919 - acc: 0.9746 - val_loss: 0.0588 - val_acc: 0.9794
Epoch 3/10
4457/4457 [==============================] - 5s - loss: 0.0568 - acc: 0.9834 - val_loss: 0.0486 - val_acc: 0.9865
Epoch 4/10
4457/4457 [==============================] - 6s - loss: 0.0423 - acc: 0.9888 - val_loss: 0.0607 - val_acc: 0.9776
Epoch 5/10
4457/4457 [==============================] - 5s - loss: 0.0335 - acc: 0.9921 - val_loss: 0.0703 - val_acc: 0.9767
Epoch 6/10
4457/4457 [==============================] - 5s - loss: 0.0318 - acc: 0.9910 - val_loss: 0.0636 - val_acc: 0.9812
Epoch 7/10
4457/4457 [==============================] - 6s - loss: 0.0308 - acc: 0.9921 - val_loss: 0.0558 - val_acc: 0.9848
Epoch 8/10
4457/4457 [==============================] - 6s - loss: 0.0266 - a

In [14]:
import matplotlib.pyplot as plt
epochs = range(1, 11)
acc = history.history['acc']
val_acc = history.history['val_acc']

plt.plot(epochs, acc, 'b+', label='Acc')
plt.plot(epochs, val_acc, 'bo', label='Val Acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()